## SITCOM Laser Tracker Analysis: M2 and Camera Offsets comparison in elevation

In this notebook we compare the laser tracker offsets against elevation for different dates.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from astropy.time import Time, TimeDelta
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

from lsst_efd_client import EfdClient
from lsst.sitcom.vandv.lasertracker import utils

%matplotlib inline
%reload_ext lab_black
%reload_ext autoreload
%autoreload 2

## Retrieve offsets from three different dates

In [ ]:
client = EfdClient('usdf_efd')
fields = ['dX', 'dY', 'dZ', 'dRX', 'dRY', 'target']

# March 10th, 2024
start = Time("2024-03-10 19:48:00Z", scale='utc')
end = Time("2024-03-10 22:20:00Z", scale='utc')

offsets_mar09 = await client.select_time_series(
    'lsst.sal.LaserTracker.logevent_offsetsPublish',
    fields,  
    start, 
    end,
)  

# July 5th, 2023
start = Time('2023-07-05 6:38:0Z', scale='utc')
end = Time('2023-07-05 10:40:0Z', scale='utc')

offsets_jul05 = await client.select_time_series(
    'lsst.sal.LaserTracker.logevent_offsetsPublish',
    fields,  
    start, 
    end,
)  

# January 12th, 2024
start = Time('2024-01-12 20:30:0Z', scale='utc')
end = Time('2024-01-12 21:15:0Z', scale='utc')

offsets_jan13 = await client.select_time_series(
    'lsst.sal.LaserTracker.logevent_offsetsPublish',
    fields,  
    start, 
    end,
)  

In [ ]:
# Extract offsets, target elevation, azimuth and rotator angle for M2
data_jul05_m2 = utils.extract_component_data(offsets_jul05, 'M2')
data_jan13_m2 = utils.extract_component_data(offsets_jan13, 'M2')
data_mar09_m2 = utils.extract_component_data(offsets_mar09, 'M2')

# Extract offsets, target elevation, azimuth and rotator angle for Camera
data_jul05_cam = utils.extract_component_data(offsets_jul05, 'Camera')
data_jan13_cam = utils.extract_component_data(offsets_jan13, 'Camera')
data_mar09_cam = utils.extract_component_data(offsets_mar09, 'Camera')

## Plot comparison of offsets for different components

### Camera

In [ ]:
plt.figure(figsize = (10, 6))
for idx, field in enumerate(fields[:-1]):
    plt.subplot(2,3,idx + 1)
    plt.plot(data_jul05_cam['target_elevation'], data_jul05_cam[field]- np.mean(data_jul05_cam[field]), '.-', label = 'July 05')
    plt.plot(data_jan13_cam['target_elevation'], data_jan13_cam[field]- np.mean(data_jan13_cam[field]), '.-', label = 'Jan 13')
    plt.plot(data_mar09_cam['target_elevation'], data_mar09_cam[field]- np.mean(data_mar09_cam[field]), '.-', label = 'Mar 09')

    # Format plot
    plt.xlabel('Elevation (deg)')
    if idx >= 3:
        plt.ylabel('deg')
    else:
        plt.ylabel('um')
    plt.title(f'Camera {field} Offsets')
    plt.legend()
    
plt.tight_layout()
plt.savefig('Comparison_Camera_Offsets.png', dpi = 300)

### M2

In [ ]:
plt.figure(figsize = (10, 6))
for idx, field in enumerate(fields[:-1]):
    plt.subplot(2,3,idx + 1)
    plt.plot(data_jul05_m2['target_elevation'], data_jul05_m2[field]- np.mean(data_jul05_m2[field]), '.-', label = 'July 05')
    plt.plot(data_jan13_m2['target_elevation'], data_jan13_m2[field]- np.mean(data_jan13_m2[field]), '.-', label = 'Jan 13')
    plt.plot(data_mar09_m2['target_elevation'], data_mar09_m2[field]- np.mean(data_mar09_m2[field]), '.-', label = 'Mar 09')

    # Format plot
    plt.xlabel('Elevation (deg)')
    if idx >= 3:
        plt.ylabel('deg')
    else:
        plt.ylabel('um')
    plt.title(f'M2  {field} Offsets')
    plt.legend()
    
plt.tight_layout()
plt.savefig('Comparison_M2_Offsets.png', dpi = 300)